In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
        builder. \
        config('spark.ui.port','0'). \
        config('spark.shuffle.useOldFetchProtocol','true'). \
        config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
        enableHiveSupport(). \
        master('yarn'). \
        getOrCreate()

In [2]:
raw_df=spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [3]:
raw_df.createOrReplaceTempView("lending_club_data")

In [4]:
from pyspark.sql.functions import sha2,concat_ws

In [5]:
new_df = raw_df.withColumn("name_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]),256))

In [6]:
new_df

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,name_sha2
68407277,null,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723916800001,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,null,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,null,null,null,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null,6d5091b3fcaaeb4ea...
68355089,null,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.66,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,null,Mar-2019,699.0,695.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,null,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,null,nul

In [7]:
new_df.createOrReplaceTempView("newtable")

In [ ]:
spark.sql(''' select name_sha2 as member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,'USA' as country,grade,sub_grade,verification_status,
tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint from newtable
''').repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv006222/myproject/raw/customers_data_csv") \
.save()

In [8]:
customers_data_df=spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/user/itv006222/myproject/raw/customers_data_csv")

In [9]:
customers_data_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
ebe2fa66486c78fda...,null,null,MORTGAGE,42000.0,LA,713xx,USA,C,C3,Source Verified,3400.0,Individual,null,null
f21094e261c23c3a4...,century concrete,10+ years,RENT,30000.0,VA,236xx,USA,C,C4,Not Verified,20200.0,Individual,null,null
814b80a446d00abd4...,Caddell Construct...,10+ years,RENT,85000.0,AL,351xx,USA,C,C2,Verified,58199.0,Individual,null,null
259a8a3d43849e98e...,Lexington family ...,10+ years,MORTGAGE,114107.0,NC,272xx,USA,A,A1,Verified,336489.0,Individual,null,null
da69229e46e25f2b3...,McKenna Brothers ...,10+ years,MORTGAGE,50000.0,FL,338xx,USA,D,D1,Source Verified,122916.0,Individual,null,null
785dd25db26ea1874...,Lorillard Tobacco...,6 years,MORTGAGE,57000.0,NC,270xx,USA,B,B3,Verified,272372.0,Individual,null,null
b0171bed5ad8037c2...,Almo Sounds Inc.,2 years,RENT,50000.0,CA,900xx,USA,B,B3,Not Verified,63476.0,Individual,null,null
00efd881340dc1344...,MoÃ«t Hennessy USA,8 years,RENT,97000.0,NY,115xx,USA,C,C5,Verified,72852.0,Individual,null,null
9d96fe1a364bcb6f8...,Alerus Financial,6 years,RENT,25000.0,MN,567xx,USA,B,B2,Not Verified,24270.0,Individual,null,null
ce3eac30eacdbf0c4...,Seattle Police De...,10+ years,OWN,54704.0,WA,983xx,USA,B,B4,Verified,96551.0,Individual,null,null


In [ ]:
spark.sql(''' select id as loan_id, name_sha2 as member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
from newtable
''').repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv006222/myproject/raw/loans_data_csv") \
.save()

In [10]:
loans_df=spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/user/itv006222/myproject/raw/loans_data_csv")

In [11]:
loans_df

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
68407277,6d5091b3fcaaeb4ea...,3600.0,3600.0,36 months,13.99,123.03,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation
68355089,b5e7938b0a2da4cea...,24700.0,24700.0,36 months,11.99,820.28,Dec-2015,Fully Paid,small_business,Business
68341763,91060b858433e8a61...,20000.0,20000.0,60 months,10.78,432.66,Dec-2015,Fully Paid,home_improvement,null
66310712,cab1fa9f533688b0a...,35000.0,35000.0,60 months,14.85,829.9,Dec-2015,Current,debt_consolidation,Debt consolidation
68476807,f74e401c1ab0adf78...,10400.0,10400.0,60 months,22.45,289.91,Dec-2015,Fully Paid,major_purchase,Major purchase
68426831,8aef4bb29d609d8d6...,11950.0,11950.0,36 months,13.44,405.18,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation
68476668,538b4653da3b1e814...,20000.0,20000.0,36 months,9.17,637.58,Dec-2015,Fully Paid,debt_consolidation,Debt consolidation
67275481,b24d55f21390533c5...,20000.0,20000.0,36 months,8.49,631.26,Dec-2015,Fully Paid,major_purchase,Major purchase
68466926,1035c5401b0ca76d0...,10000.0,10000.0,36 months,6.49,306.45,Dec-2015,Fully Paid,credit_card,Credit card refin...
68616873,cb0f1777593e77909...,8000.0,8000.0,36 months,11.48,263.74,Dec-2015,Fully Paid,credit_card,Credit card refin...


In [ ]:
spark.sql(''' select 
id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
from newtable
''').repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv006222/myproject/raw/loans_repayments_csv") \
.save()

In [12]:
loans_repayments_df=spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/user/itv006222/myproject/raw/loans_repayments_csv")

In [13]:
loans_repayments_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
68407277,3600.0,821.72,0.0,4421.723916800001,122.67,Jan-2019,null
68355089,24700.0,979.66,0.0,25679.66,926.35,Jun-2016,null
68341763,20000.0,2705.92,0.0,22705.924293878397,15813.3,Jun-2017,null
66310712,19102.35,12361.66,0.0,31464.01,829.9,Feb-2019,Apr-2019
68476807,10400.0,1340.5,0.0,11740.5,10128.96,Jul-2016,null
68426831,11950.0,1758.95,0.0,13708.9485297572,7653.56,May-2017,null
68476668,20000.0,1393.8,0.0,21393.800000011,15681.05,Nov-2016,null
67275481,20000.0,1538.51,0.0,21538.508976797,14618.23,Jan-2017,null
68466926,10000.0,998.97,0.0,10998.9715749644,1814.48,Aug-2018,null
68616873,8000.0,939.58,0.0,8939.5805031401,4996.24,Apr-2017,null


In [ ]:
spark.sql(''' select name_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record 
from newtable
''').repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv006222/myproject/raw/loans_defaulters_csv") \
.save()

In [15]:
loans_defaulters_df=spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/user/itv006222/myproject/raw/loans_defaulters_csv")

In [16]:
loans_defaulters_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
b59d80da191f5b573...,0.0,0.0,0.0,0.0,1.0,0.0,31.0,null
202d9f56ecb7c3bc9...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,null
e5a140c0922b554b9...,0.0,0.0,0.0,0.0,0.0,0.0,47.0,null
e12aefc548f750777...,0.0,0.0,0.0,0.0,0.0,0.0,33.0,null
1b3a50d854fbbf97e...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
1c4329e5f17697127...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
5026c86ad983175eb...,0.0,0.0,1.0,0.0,2.0,0.0,null,71.0
9847d8c1e9d0b2084...,1.0,0.0,2.0,0.0,0.0,0.0,6.0,63.0
8340dbe1adea41fb4...,0.0,0.0,0.0,0.0,0.0,0.0,36.0,null
d4de0de3ab7d79ad4...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,null
